In [ ]:
!pip install nest-asyncio pyngrok fastapi uvicorn python-multipart

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration, AutoTokenizer, ElectraModel
import tensorflow as tf
import re

from fastapi import FastAPI, File, UploadFile, WebSocket, WebSocketDisconnect
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from tempfile import NamedTemporaryFile
import uvicorn
from pydantic import BaseModel
from typing import List

import nest_asyncio
from pyngrok import ngrok

import torch
import torchaudio
import os
import openai
import numpy as np
import time

In [ ]:
# 파인튜닝된 whisper 모델 로드
lora_model_name = "/content/drive/MyDrive/whisper_m__250512_001"  # LoRA 모델 경로
lora_model = WhisperForConditionalGeneration.from_pretrained(lora_model_name)
lora_processor = WhisperProcessor.from_pretrained(lora_model_name)
# 추론 모드로 전환
lora_model.eval()

# 긴급도 분류 모델 로드
urgency_model = tf.keras.models.load_model("/content/drive/MyDrive/urgency_classification_001.h5")

# 긴급도 분류 모델을 위한 tokenizer 및 임베딩 모델 로드
urgency_tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
urgency_embedding_model = ElectraModel.from_pretrained("beomi/KcELECTRA-base")
urgency_embedding_model.eval()

# 텍스트 정제, 토큰화, 임베딩
def text_processing(text: str):
    stopwords = ["개인정보"]
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(\b\w+\b)( \1)+", r"\1", text)
    words = text.split()
    words = [word for word in words if word not in stopwords]

    clean_text = ' '.join(words)

    inputs = urgency_tokenizer(
        clean_text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = urgency_embedding_model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]

    return cls_embedding.squeeze().numpy().tolist()

# 긴급도 분류
def classify_urgency(text, disaster_type):
    text_input = text_processing(text)  # 임베딩
    text_input = np.array(text_input).reshape(1, -1)  # 2D 배열로 변환

    disaster_type = np.array(disaster_type).reshape(1, -1)  # 2D로 변환

    # 예측
    pred = urgency_model.predict({"text_input": text_input, "disaster_input": disaster_type})
    pred_class = np.argmax(pred, axis=1)[0]

    # 클래스 인덱스 상중하로 매핑
    urgency_label = {0: '하', 1: '중', 2: '상'}
    return urgency_label[pred_class]


In [ ]:
import getpass
import openai
openai.api_key = getpass.getpass(prompt = 'OpenAI API키 입력')

OpenAI API키 입력··········


In [ ]:
app = FastAPI(
    title="VoiceFront119",
    version="1.0",
    decsription="API Server"
)

origins = ["*"]

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

# FastAPI 연계 확인
class FileInput(BaseModel):
    file_path: str

class UrgencyInput(BaseModel):
    text: str
    disaster_type: List[int]

class GptInput(BaseModel):
  text: str

# 파인튜닝된 Whisper 모델로 STT 결과 반환
# 전체 wav 파일 20초 단위로 잘라서 텍스트 변환
@app.post("/stt_result", status_code=200)
async def stt_from_audio(file: UploadFile = File(...)):
    try:
        # file_path = input.file_path
        print("Received file:", file.filename)

        # 임시 파일로 저장
        with NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
            tmp.write(await file.read())
            tmp_path = tmp.name

        # torchaudio로 오디오 로딩
        speech_array, original_sampling_rate = torchaudio.load(tmp_path)
        os.remove(tmp_path)

        sampling_rate = 16000

        # 16kHz로 리샘플링
        if original_sampling_rate != sampling_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=sampling_rate)
            speech_array = resampler(speech_array)

        chunk_duration = 20 # sec
        chunk_size = chunk_duration * sampling_rate
        full_transcription = []

        # 20초 단위로 슬라이딩
        for start in range(0, speech_array.shape[1], chunk_size):
            end = start + chunk_size
            chunk = speech_array[:, start:end]

            # 너무 짧은 chunk는 생략
            if chunk.shape[1] < sampling_rate * 2:
                continue

            # 입력 처리
            inputs = lora_processor(
                chunk.squeeze(),
                sampling_rate=sampling_rate,
                return_tensors="pt",
                return_attention_mask=True
            )

            # 추론
            with torch.no_grad():
                predicted_ids = lora_model.generate(
                    inputs["input_features"],
                    attention_mask=inputs["attention_mask"],
                    language="ko"
                )

            # 디코딩
            transcription = lora_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
            full_transcription.append(transcription)

        stt_result = " ".join(full_transcription)

        return {"stt_result": stt_result}
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)


# # 클라이언트로부터 잘라진 wav 파일 받아와서 지속적으로 변환
# @app.websocket("/ws_stt")
# async def websocket_endpoint(websocket: WebSocket):
#     await websocket.accept()

#     try:
#         while True:
#             data = await websocket.receive_bytes()
#             with NamedTemporaryFile(delete=False, suffix=".wav") as tmp:
#                 tmp.write(data)
#                 tmp_path = tmp.name

#             # 여기서 STT 수행
#             result = wav_to_text(tmp_path)
#             os.remove(tmp_path)

#             await websocket.send_text(result)
#     except Exception as e:
#         print("연결 종료:", e)
#     finally:
#         await websocket.close()


# 긴급도 분류
@app.post("/urgency_classification")
async def urgency_classification(input: UrgencyInput):
    print('Input received.')
    try:
        result = classify_urgency(input.text, input.disaster_type)
        return {"urgency_level": result}
    except Exception as e:
        return JSONResponse(content={"error": str(e)}, status_code=500)

# 신고내용 요약 및 분석
@app.post("/gpt_response")
async def gpt4_turbo_response(input: GptInput):
    print('Input received.')
    response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[{"role": "system",
                       "content":"""
                        당신은 119 대원들의 신고 접수를 돕는 시스템이다.
                        신고 내용을 기반으로 핵심 정보를 요약하고 119 출동 대원이 빠르게 파악할 수 있도록 정리해야 한다.

                        작성 지침 :
                        - 모든 항목은 아이템 형태(• 또는 -)로 간결하게 작성한다.
                        - 신고 위치와 신고 내용은 반드시 포함되어야 한다.
                        - 신고 내용의 첫번째 줄에서는 문제 상황을 한줄로 간단히 정의한다.

                        - 구조/구급 상황일 경우:
                          - 환자와 신고자의 관계
                          - 환자의 상태 또는 증상 등
                        - 화재 상황일 경우:
                          - 화재 발생 위치
                          - 현재 진행 상황 (연기, 화염, 대피 여부 등) 등

                        - 주어지지 않은 정보를 허위로 작성해선 안 된다.
                        - 그 외 유의사항이나 추가 확인이 필요한 정보는 "비고" 항목에 명확히 정리한다.
                        - 모든 내용은 한국어로 작성하고, 출동 대원이 즉시 파악할 수 있도록 한다.
                        - 볼드나 기울임 등 마크다운은 사용하지 않는다.
                        """
                       },
                      {"role": "user",
                       "content": input.text
                       }],
            # max_tokens=2000,
            temperature=0.7,
            frequency_penalty=0.7
        )

    gpt_response = response.choices[0].message.content.strip()
    return {"gpt_response": gpt_response}


@app.get("/")
async def root():
    return {"message": "VoiceFront119 Model API is running"}

In [ ]:
# ngrok 인증 토큰과 포트 설정
auth_token = "ngrok 인증 토큰 입력"
ngrok.set_auth_token(auth_token)
# ngrok 연결
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
# 비동기 처리를 위한 nest_asyncio 적용
nest_asyncio.apply()
uvicorn.run(app, port=8000, timeout_keep_alive=300)

공용 URL https://c0e4-35-204-127-207.ngrok-free.app


INFO:     Started server process [608]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Received file: 광주_화재.wav
INFO:     218.147.225.131:0 - "POST /stt_result HTTP/1.1" 200 OK
Input received.
INFO:     218.147.225.131:0 - "POST /gpt_response HTTP/1.1" 200 OK
Input received.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
INFO:     218.147.225.131:0 - "POST /urgency_classification HTTP/1.1" 200 OK
Received file: 광주_화재.wav
INFO:     218.147.225.131:0 - "POST /stt_result HTTP/1.1" 200 OK
Input received.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
INFO:     218.147.225.131:0 - "POST /urgency_classification HTTP/1.1" 200 OK
Input received.
INFO:     218.147.225.131:0 - "POST /gpt_response HTTP/1.1" 200 OK
Received file: 광주_화재.wav
INFO:     218.147.225.131:0 - "POST /stt_result HTTP/1.1" 200 OK
Input received.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
INFO:     218.147.225.131:0 - "POST /urgency_classification HTTP/1.1" 200 OK
Input received.
INFO:     218.147.225.131:0 - "POST /gpt_response HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [608]
